# Chapter 06: 메모리 (Memory)

이 노트북에서는 에이전트의 단기/장기 메모리 구현을 학습합니다.

## 주요 내용
- 단기 메모리
- 메모리 유지 대화
- 체크포인터 활용

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/TeeDDub/Building-Applications-with-AI-Agents/blob/main/notebook/ch06_memory.ipynb)


## 1. 패키지 설치


In [ ]:
!pip install -q langchain langchain-openai langgraph python-dotenv rank-bm25


## 2. API 키 설정


In [ ]:
import os

try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ Colab Secrets에서 API 키를 불러왔습니다.")
except:
    pass

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "sk-your-api-key-here"
    print("⚠️ API 키를 직접 입력해주세요.")


## 3. short_term_memory.py


메모리 유무에 따른 대화 차이를 보여줍니다.


In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, MessagesState, START

# 환경변수 확인
import os
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError(
        "OPENAI_API_KEY가 설정되지 않았습니다."
        "환경변수 또는 .env 파일에서 설정해주세요."
    )

# LLM 초기화
llm = init_chat_model(model="gpt-5-mini", temperature=0)


def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}


builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_edge(START, "call_model")
graph = builder.compile()

# 메모리가 없어서 대화 상태를 유지할 수 없음
input_message = {"type": "user", "content": "안녕하세요! 제 이름은 민혁입니다."}
for chunk in graph.stream({"messages": [input_message]}, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

input_message = {"type": "user", "content": "제 이름이 뭐라고 했죠?"}
for chunk in graph.stream({"messages": [input_message]}, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

from langgraph.checkpoint.memory import MemorySaver

# 메모리가 유지되어 대화 상태를 유지할 수 있음
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}
input_message = {"type": "user", "content": "안녕하세요! 제 이름은 민혁입니다."}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()



## 4. basic_bm25.py


BM25로 간단한 텍스트 검색을 수행합니다.


In [ ]:
from rank_bm25 import BM25Okapi
from typing import List

corpus: List[List[str]] = [
    "에이전트 J는 패기가 넘치는 신입 대원이다".split(),
    "에이전트 K는 수년간의 MIB 경험과 멋진 뉴럴라이저를 갖고 있다".split(),
    "두 명의 에이전트가 검은 정장을 입고 은하계를 구했다".split(),
]
# 2. BM25 인덱스 생성
bm25 = BM25Okapi(corpus)

# 3. 간단한 쿼리로 검색 수행
query = "신입 대원은 누구지?".split()
top_n = bm25.get_top_n(query, corpus, n=2)

print("쿼리:", " ".join(query))
print("상위 일치 문장:")
for line in top_n:
    print(" •", " ".join(line))
